<h1>CycleGAN for Style Transfer</h1>
<h3> Imports </h3>

In [9]:
#Python imports
import torch
import torch.nn as tnn
from torch.autograd import Variable
import sys
import itertools
from torch.utils.data import DataLoader
from Dataset import ImageSet
import datetime
import torchvision.utils as tv
import os
import matplotlib.pyplot as plt
import numpy as np

#Import network elements
import network_elements as net

#stdout code blocks reload's print statement
#prevstd = sys.stdout
#sys.stdout = None
reload(net)
print("Reloading net")
#sys.stdout = prevstd


Reloading net


<h3>Set parameters</h3>

In [10]:
#The number of channels in the input image
im_in_channels = 3

#The number of channels in the output image
im_out_channels = 3

#The size of the largest size of the input image
im_size = 256

LEARNING_RATE = 0.0002

LR_DECAY_START = 100
LR_DECAY_END = 200
NUM_EPOCHS = 200

<h3>Import dataset</h3>

In [11]:
imageSet = ImageSet();
dataset = "horse2zebra"
imageSet.loadData(dataset, 'train', im_size)
imgLoader = DataLoader(imageSet,shuffle=True)

./datasets/horse2zebra
Finished loading data


<h3>Define the CycleGAN network</h3>

In [12]:
#G : X->Y
G = net.Mapping(im_in_channels, im_out_channels, im_size)

#F : Y->X
F = net.Mapping(im_out_channels, im_out_channels, im_size)

#G mapping discriminator
Dx = net.Discriminator(im_in_channels)

#F mapping discriminator
Dy = net.Discriminator(im_out_channels)

#Initialize CUDA implementations
G.cuda();
F.cuda();
Dx.cuda();
Dy.cuda();

#Define losses
cycle_loss = tnn.L1Loss(); #Cycle-consistency loss
gan_loss = tnn.MSELoss(); #Adversarial loss
identity_loss = tnn.L1Loss(); #Prevents image tinting

<h3>Initialize weights for each network</h3>

In [13]:
# norm_weights function applies Gaussian norm weights with mean 0 and stddev 0.02
G.apply(net.conv_norm_weights);
F.apply(net.conv_norm_weights);
Dx.apply(net.conv_norm_weights);
Dy.apply(net.conv_norm_weights);

<h3>Set optimizer parameters</h3>

In [14]:
#simplified version has no pooling, paper does not, paper code does...
# REMOVE

gan_opt = torch.optim.Adam(itertools.chain(G.parameters(), F.parameters()),
                           lr = LEARNING_RATE)
dx_opt = torch.optim.Adam(Dx.parameters(), lr = LEARNING_RATE)
dy_opt = torch.optim.Adam(Dy.parameters(), lr = LEARNING_RATE)

gan_lr = torch.optim.lr_scheduler.LambdaLR(gan_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step)
dx_lr = torch.optim.lr_scheduler.LambdaLR(dx_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step)
dy_lr = torch.optim.lr_scheduler.LambdaLR(dy_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step)

The paper uses Adam optimizer with a learning rate of 0.0002 for the first 100 epochs that linearly decays to 0 over the next 100 epochs.

<h3> Initialize training variables </h3>

In [15]:
x = torch.cuda.FloatTensor(1, im_in_channels, im_size, im_size)
y = torch.cuda.FloatTensor(1, im_out_channels, im_size, im_size)

# Variable wrapper is depricated in >0.4
dReal = Variable(torch.cuda.FloatTensor(1).fill_(1.0), requires_grad = False)
dFake = Variable(torch.cuda.FloatTensor(1).fill_(0.0), requires_grad = False)

buffX_Y = net.MapBuffer()
buffY_X = net.MapBuffer()

if not os.path.isdir("./img/"):
    os.mkdir("./img/")
    
if not os.path.isdir("./log/"):
    os.mkdir("./log/")
    
if os.path.isfile("./log/loss.log"):    
    os.remove("./log/loss.log")
    
logfile = open("./log/loss.log","a")

<h3> Training </h3>

In [ ]:
start_time = datetime.datetime.now()
for i in range(0,NUM_EPOCHS):
    for j, img in enumerate(imgLoader):
        elapsed_time = datetime.datetime.now()-start_time
        print("Epoch %d/%d, %d/%d, %s"%(i,LR_DECAY_END,j,len(imageSet),str(elapsed_time)))
        src_x = Variable(x.copy_(img['x']))
        src_y = Variable(y.copy_(img['y']))
        
        gan_opt.zero_grad()
        
        #identity Loss
        mapY_Y = G(src_y)
        lossY_Y = identity_loss(mapY_Y, src_y)*5.0
        
        mapX_X = F(src_x)
        lossX_X = identity_loss(mapX_X, src_x)*5.0
        
        #GAN Loss
        mapX_Y = G(src_x)
        dX_Y = Dx(mapX_Y)
        lossX_Y = gan_loss(dX_Y, dReal)
        
        mapY_X = F(src_y)
        dY_X = Dy(mapY_X)
        lossY_X = gan_loss(dY_X, dReal)
        
        #Cycle Consistency Loss
        mapX_Y_X = F(mapX_Y)
        lossX_Y_X = cycle_loss(mapX_Y_X, src_x)*10.0
        
        mapY_X_Y = G(mapY_X)
        lossY_X_Y = cycle_loss(mapY_X_Y, src_y)*10.0
        
        mapLoss = lossY_Y + lossX_X + lossX_Y + lossY_X + lossX_Y_X + lossY_X_Y
        mapLoss.backward()
        
        gan_opt.step()
        
        #Dx
        dx_opt.zero_grad()
        
        dX_res = Dx(src_x)
        lossX = gan_loss(dX_res, dReal)
        
        mapY_X = buffY_X.cycle(mapY_X)
        dY_X = Dx(mapY_X.detach())
        lossY_X = gan_loss(dY_X, dFake)
        
        dxLoss = (lossX + lossY_X)*0.5
        dxLoss.backward()
        
        dx_opt.step()
        
        #Dy
        dy_opt.zero_grad()
        
        dY_res = Dy(src_y)
        lossY = gan_loss(dY_res, dReal)
        
        mapX_Y = buffX_Y.cycle(mapX_Y)
        dX_Y = Dy(mapX_Y.detach())
        lossX_Y = gan_loss(dX_Y, dFake)
        
        dyLoss = (lossY + lossX_Y)*0.5
        dyLoss.backward()
        
        dy_opt.step()     
        
        
        
        logfile.write("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
    plt.imshow(np.transpose(torch.squeeze(mapX_Y.data.cpu()).numpy(), (1,2,0)), interpolation='nearest')
    tv.save_image(src_x.data, './img/src_x_%d.jpg'%(i))
    tv.save_image(src_y.data, './img/src_x_%d.jpg'%(i))
    tv.save_image(mapX_Y.data, './img/mapX_Y_%d.jpg'%(i))
    tv.save_image(mapY_X.data, './img/mapY_X_%d.jpg'%(i))
    gan_lr.step()
    dx_lr.step()
    dy_lr.step()
    
    if (i%5 == 0):
        if not os.path.isdir("./model/"):
            os.mkdir("./model/")
        torch.save(G.state_dict(), './model/G.data')
        torch.save(F.state_dict(), './model/F.data')
        torch.save(Dx.state_dict(), './model/Dx.data')
        torch.save(Dy.state_dict(), './model/Dy.data')

Epoch 0/200, 0/1334, 0:00:00.012291
Epoch 0/200, 1/1334, 0:00:00.682081
Epoch 0/200, 2/1334, 0:00:01.270850
Epoch 0/200, 3/1334, 0:00:01.875665
Epoch 0/200, 4/1334, 0:00:02.477762
Epoch 0/200, 5/1334, 0:00:03.075658
Epoch 0/200, 6/1334, 0:00:03.688723
Epoch 0/200, 7/1334, 0:00:04.282253
Epoch 0/200, 8/1334, 0:00:04.872951
Epoch 0/200, 9/1334, 0:00:05.476147
Epoch 0/200, 10/1334, 0:00:06.066705
Epoch 0/200, 11/1334, 0:00:06.665629
Epoch 0/200, 12/1334, 0:00:07.278969
Epoch 0/200, 13/1334, 0:00:07.875177
Epoch 0/200, 14/1334, 0:00:08.470734
Epoch 0/200, 15/1334, 0:00:09.082682
Epoch 0/200, 16/1334, 0:00:09.678699
Epoch 0/200, 17/1334, 0:00:10.276535
Epoch 0/200, 18/1334, 0:00:10.877155
Epoch 0/200, 19/1334, 0:00:11.481635
Epoch 0/200, 20/1334, 0:00:12.073248
Epoch 0/200, 21/1334, 0:00:12.681387
Epoch 0/200, 22/1334, 0:00:13.281217
Epoch 0/200, 23/1334, 0:00:13.878183
Epoch 0/200, 24/1334, 0:00:14.480304
Epoch 0/200, 25/1334, 0:00:15.074379
Epoch 0/200, 26/1334, 0:00:15.689146
Epoch 0/200

Epoch 0/200, 219/1334, 0:02:11.479038
Epoch 0/200, 220/1334, 0:02:12.071512
Epoch 0/200, 221/1334, 0:02:12.684458
Epoch 0/200, 222/1334, 0:02:13.282297
Epoch 0/200, 223/1334, 0:02:13.876887
Epoch 0/200, 224/1334, 0:02:14.474828
Epoch 0/200, 225/1334, 0:02:15.084665
Epoch 0/200, 226/1334, 0:02:15.691340
Epoch 0/200, 227/1334, 0:02:16.286488
Epoch 0/200, 228/1334, 0:02:16.890228
Epoch 0/200, 229/1334, 0:02:17.488782
Epoch 0/200, 230/1334, 0:02:18.102247
Epoch 0/200, 231/1334, 0:02:18.730724
Epoch 0/200, 232/1334, 0:02:19.299506
Epoch 0/200, 233/1334, 0:02:19.869903
Epoch 0/200, 234/1334, 0:02:20.485849
Epoch 0/200, 235/1334, 0:02:21.090107
Epoch 0/200, 236/1334, 0:02:21.675539
Epoch 0/200, 237/1334, 0:02:22.275484
Epoch 0/200, 238/1334, 0:02:22.873792
Epoch 0/200, 239/1334, 0:02:23.476768
Epoch 0/200, 240/1334, 0:02:24.082858
Epoch 0/200, 241/1334, 0:02:24.684984
Epoch 0/200, 242/1334, 0:02:25.289197
Epoch 0/200, 243/1334, 0:02:25.882164
Epoch 0/200, 244/1334, 0:02:26.489648
Epoch 0/200,

Epoch 0/200, 435/1334, 0:04:21.087868
Epoch 0/200, 436/1334, 0:04:21.692511
Epoch 0/200, 437/1334, 0:04:22.281466
Epoch 0/200, 438/1334, 0:04:22.889299
Epoch 0/200, 439/1334, 0:04:23.484485
Epoch 0/200, 440/1334, 0:04:24.079366
Epoch 0/200, 441/1334, 0:04:24.683783
Epoch 0/200, 442/1334, 0:04:25.286564
Epoch 0/200, 443/1334, 0:04:25.883088
Epoch 0/200, 444/1334, 0:04:26.498420
Epoch 0/200, 445/1334, 0:04:27.087378
Epoch 0/200, 446/1334, 0:04:27.668035
Epoch 0/200, 447/1334, 0:04:28.285534
Epoch 0/200, 448/1334, 0:04:28.888632
Epoch 0/200, 449/1334, 0:04:29.482916
Epoch 0/200, 450/1334, 0:04:30.086492
Epoch 0/200, 451/1334, 0:04:30.693622
Epoch 0/200, 452/1334, 0:04:31.285832
Epoch 0/200, 453/1334, 0:04:31.889324
Epoch 0/200, 454/1334, 0:04:32.483749
Epoch 0/200, 455/1334, 0:04:33.090931
Epoch 0/200, 456/1334, 0:04:33.677307
Epoch 0/200, 457/1334, 0:04:34.271050
Epoch 0/200, 458/1334, 0:04:34.875632
Epoch 0/200, 459/1334, 0:04:35.498728
Epoch 0/200, 460/1334, 0:04:36.092814
Epoch 0/200,

Epoch 0/200, 651/1334, 0:06:30.678573
Epoch 0/200, 652/1334, 0:06:31.280598
Epoch 0/200, 653/1334, 0:06:31.874649
Epoch 0/200, 654/1334, 0:06:32.473188
Epoch 0/200, 655/1334, 0:06:33.086538
Epoch 0/200, 656/1334, 0:06:33.684108
Epoch 0/200, 657/1334, 0:06:34.282001
Epoch 0/200, 658/1334, 0:06:34.882134
Epoch 0/200, 659/1334, 0:06:35.485769
Epoch 0/200, 660/1334, 0:06:36.080020
Epoch 0/200, 661/1334, 0:06:36.682065
Epoch 0/200, 662/1334, 0:06:37.274403
Epoch 0/200, 663/1334, 0:06:37.882770
Epoch 0/200, 664/1334, 0:06:38.479050
Epoch 0/200, 665/1334, 0:06:39.084463
Epoch 0/200, 666/1334, 0:06:39.672116
Epoch 0/200, 667/1334, 0:06:40.273705
Epoch 0/200, 668/1334, 0:06:40.875165
Epoch 0/200, 669/1334, 0:06:41.468054
Epoch 0/200, 670/1334, 0:06:42.070320
Epoch 0/200, 671/1334, 0:06:42.675907
Epoch 0/200, 672/1334, 0:06:43.275266
Epoch 0/200, 673/1334, 0:06:43.870614
Epoch 0/200, 674/1334, 0:06:44.481912
Epoch 0/200, 675/1334, 0:06:45.087521
Epoch 0/200, 676/1334, 0:06:45.683161
Epoch 0/200,

Epoch 0/200, 867/1334, 0:08:40.274659
Epoch 0/200, 868/1334, 0:08:40.871280
Epoch 0/200, 869/1334, 0:08:41.473798
Epoch 0/200, 870/1334, 0:08:42.070656
Epoch 0/200, 871/1334, 0:08:42.676084
Epoch 0/200, 872/1334, 0:08:43.281688
Epoch 0/200, 873/1334, 0:08:43.890612
Epoch 0/200, 874/1334, 0:08:44.485447
Epoch 0/200, 875/1334, 0:08:45.087192
Epoch 0/200, 876/1334, 0:08:45.680744
Epoch 0/200, 877/1334, 0:08:46.289897
Epoch 0/200, 878/1334, 0:08:46.880454
Epoch 0/200, 879/1334, 0:08:47.481047
Epoch 0/200, 880/1334, 0:08:48.090323
Epoch 0/200, 881/1334, 0:08:48.679223
Epoch 0/200, 882/1334, 0:08:49.296017
Epoch 0/200, 883/1334, 0:08:49.884264
Epoch 0/200, 884/1334, 0:08:50.482493
Epoch 0/200, 885/1334, 0:08:51.083662
Epoch 0/200, 886/1334, 0:08:51.684491
Epoch 0/200, 887/1334, 0:08:52.290173
Epoch 0/200, 888/1334, 0:08:52.888385
Epoch 0/200, 889/1334, 0:08:53.487820
Epoch 0/200, 890/1334, 0:08:54.085296
Epoch 0/200, 891/1334, 0:08:54.686468
Epoch 0/200, 892/1334, 0:08:55.283066
Epoch 0/200,

...

Epoch 4/200, 694/1334, 1:00:20.092101
Epoch 4/200, 695/1334, 1:00:20.696040
Epoch 4/200, 696/1334, 1:00:21.329339
Epoch 4/200, 697/1334, 1:00:21.896442
Epoch 4/200, 698/1334, 1:00:22.492232
Epoch 4/200, 699/1334, 1:00:23.092218
Epoch 4/200, 700/1334, 1:00:23.692067
Epoch 4/200, 701/1334, 1:00:24.330794
Epoch 4/200, 702/1334, 1:00:24.896527
Epoch 4/200, 703/1334, 1:00:25.529256
Epoch 4/200, 704/1334, 1:00:26.129763
Epoch 4/200, 705/1334, 1:00:26.699026
Epoch 4/200, 706/1334, 1:00:27.330486
Epoch 4/200, 707/1334, 1:00:27.931175
Epoch 4/200, 708/1334, 1:00:28.531302
Epoch 4/200, 709/1334, 1:00:29.129191
Epoch 4/200, 710/1334, 1:00:29.729780
Epoch 4/200, 711/1334, 1:00:30.295099
Epoch 4/200, 712/1334, 1:00:30.930130
Epoch 4/200, 713/1334, 1:00:31.531300
Epoch 4/200, 714/1334, 1:00:32.095638
Epoch 4/200, 715/1334, 1:00:32.688782
Epoch 4/200, 716/1334, 1:00:33.296215
Epoch 4/200, 717/1334, 1:00:33.903129
Epoch 4/200, 718/1334, 1:00:34.492899
Epoch 4/200, 719/1334, 1:00:35.102575
Epoch 4/200,

Epoch 4/200, 910/1334, 1:02:29.688546
Epoch 4/200, 911/1334, 1:02:30.282283
Epoch 4/200, 912/1334, 1:02:30.875413
Epoch 4/200, 913/1334, 1:02:31.468720
Epoch 4/200, 914/1334, 1:02:32.080109
Epoch 4/200, 915/1334, 1:02:32.675027
Epoch 4/200, 916/1334, 1:02:33.272324
Epoch 4/200, 917/1334, 1:02:33.882786
Epoch 4/200, 918/1334, 1:02:34.482173
Epoch 4/200, 919/1334, 1:02:35.098181
Epoch 4/200, 920/1334, 1:02:35.683218
Epoch 4/200, 921/1334, 1:02:36.277629
Epoch 4/200, 922/1334, 1:02:36.880538
Epoch 4/200, 923/1334, 1:02:37.476315
Epoch 4/200, 924/1334, 1:02:38.076319
Epoch 4/200, 925/1334, 1:02:38.683993
Epoch 4/200, 926/1334, 1:02:39.277051
Epoch 4/200, 927/1334, 1:02:39.873460
Epoch 4/200, 928/1334, 1:02:40.494059
Epoch 4/200, 929/1334, 1:02:41.083693
Epoch 4/200, 930/1334, 1:02:41.681919
Epoch 4/200, 931/1334, 1:02:42.287922
Epoch 4/200, 932/1334, 1:02:42.889893
Epoch 4/200, 933/1334, 1:02:43.489947
Epoch 4/200, 934/1334, 1:02:44.091545
Epoch 4/200, 935/1334, 1:02:44.699450
Epoch 4/200,

Epoch 4/200, 1123/1334, 1:04:37.488633
Epoch 4/200, 1124/1334, 1:04:38.086862
Epoch 4/200, 1125/1334, 1:04:38.689074
Epoch 4/200, 1126/1334, 1:04:39.291623
Epoch 4/200, 1127/1334, 1:04:39.897518
Epoch 4/200, 1128/1334, 1:04:40.470483
Epoch 4/200, 1129/1334, 1:04:41.080819
Epoch 4/200, 1130/1334, 1:04:41.685499
Epoch 4/200, 1131/1334, 1:04:42.278443
Epoch 4/200, 1132/1334, 1:04:42.878052
Epoch 4/200, 1133/1334, 1:04:43.481037
Epoch 4/200, 1134/1334, 1:04:44.090872
Epoch 4/200, 1135/1334, 1:04:44.674429
Epoch 4/200, 1136/1334, 1:04:45.277798
Epoch 4/200, 1137/1334, 1:04:45.884550
Epoch 4/200, 1138/1334, 1:04:46.484210
Epoch 4/200, 1139/1334, 1:04:47.087302
Epoch 4/200, 1140/1334, 1:04:47.679683
Epoch 4/200, 1141/1334, 1:04:48.281892
Epoch 4/200, 1142/1334, 1:04:48.883146
Epoch 4/200, 1143/1334, 1:04:49.496200
Epoch 4/200, 1144/1334, 1:04:50.088091
Epoch 4/200, 1145/1334, 1:04:50.693706
Epoch 4/200, 1146/1334, 1:04:51.298657
Epoch 4/200, 1147/1334, 1:04:51.877061
Epoch 4/200, 1148/1334, 1

Epoch 5/200, 0/1334, 1:06:44.631751
Epoch 5/200, 1/1334, 1:06:45.230432
Epoch 5/200, 2/1334, 1:06:45.831030
Epoch 5/200, 3/1334, 1:06:46.430108
Epoch 5/200, 4/1334, 1:06:47.029971
Epoch 5/200, 5/1334, 1:06:47.629596
Epoch 5/200, 6/1334, 1:06:48.229547
Epoch 5/200, 7/1334, 1:06:48.830989
Epoch 5/200, 8/1334, 1:06:49.435980
Epoch 5/200, 9/1334, 1:06:50.002120
Epoch 5/200, 10/1334, 1:06:50.590624
Epoch 5/200, 11/1334, 1:06:51.192689
Epoch 5/200, 12/1334, 1:06:51.800853
Epoch 5/200, 13/1334, 1:06:52.393166
Epoch 5/200, 14/1334, 1:06:52.997505
Epoch 5/200, 15/1334, 1:06:53.632642
Epoch 5/200, 16/1334, 1:06:54.205309
Epoch 5/200, 17/1334, 1:06:54.782825
Epoch 5/200, 18/1334, 1:06:55.389813
Epoch 5/200, 19/1334, 1:06:55.986981
Epoch 5/200, 20/1334, 1:06:56.587051
Epoch 5/200, 21/1334, 1:06:57.193742
Epoch 5/200, 22/1334, 1:06:57.799410
Epoch 5/200, 23/1334, 1:06:58.391767
Epoch 5/200, 24/1334, 1:06:59.029914
Epoch 5/200, 25/1334, 1:06:59.606079
Epoch 5/200, 26/1334, 1:07:00.231883
Epoch 5/200

Epoch 5/200, 219/1334, 1:08:55.981542
Epoch 5/200, 220/1334, 1:08:56.585861
Epoch 5/200, 221/1334, 1:08:57.187293
Epoch 5/200, 222/1334, 1:08:57.781868
Epoch 5/200, 223/1334, 1:08:58.378888
Epoch 5/200, 224/1334, 1:08:58.981753
Epoch 5/200, 225/1334, 1:08:59.586516
Epoch 5/200, 226/1334, 1:09:00.230288
Epoch 5/200, 227/1334, 1:09:00.798630
Epoch 5/200, 228/1334, 1:09:01.385510
Epoch 5/200, 229/1334, 1:09:01.983738
Epoch 5/200, 230/1334, 1:09:02.582858
Epoch 5/200, 231/1334, 1:09:03.181444
Epoch 5/200, 232/1334, 1:09:03.779382
Epoch 5/200, 233/1334, 1:09:04.381713
Epoch 5/200, 234/1334, 1:09:04.981791
Epoch 5/200, 235/1334, 1:09:05.589071
Epoch 5/200, 236/1334, 1:09:06.184378
Epoch 5/200, 237/1334, 1:09:06.778042
Epoch 5/200, 238/1334, 1:09:07.380888
Epoch 5/200, 239/1334, 1:09:07.979943
Epoch 5/200, 240/1334, 1:09:08.631338
Epoch 5/200, 241/1334, 1:09:09.193111
Epoch 5/200, 242/1334, 1:09:09.788080
Epoch 5/200, 243/1334, 1:09:10.390221
Epoch 5/200, 244/1334, 1:09:10.987307
Epoch 5/200,

In [ ]:
close(logfile)